In [ ]:
import os
import pandas as pd
import numpy as np
from gcp_utils import constants
from gcp_utils.tools.utils import format_as_json
from firebase_admin import firestore, initialize_app
from database_tools.tools.dataset import ConfigMapper
from gcp_utils.tools import preprocess, predict
from plotly.subplots import make_subplots

os.chdir('/home/cam/Documents/gcp_utils/')

pd.options.plotting.backend = 'plotly'

initialize_app()
database = firestore.client()
col = database.collection(u'bpm_data_test').document('wx1jF08b3DTPijtQcwGiEwpEFai2').collection('windows')

In [ ]:
cm = ConfigMapper(constants.CONFIG_PATH)

processed_windows = []
for i, doc in enumerate(col.stream()):
    data = doc.to_dict()
    if data['fid'] == '0':
        # print(i)
        result = preprocess.validate_window(data['ppg'], cm)
        print(result['flat_lines'])
        data.update({
            'status': result['status'],
            'vpg': result['vpg'],
            'apg': result['apg'],
            'ppg_scaled': result['ppg_scaled'],
            'vpg_scaled': result['vpg_scaled'],
            'apg_scaled': result['apg_scaled'],
            'f0': result['f0'],
            'snr': result['snr'],
            'beat_sim': result['beat_sim'],
        })

        if result['status'] == 'valid':
            result = predict.predict_bp(format_as_json(data)[0], cm)
            data.update({
                'status': 'predicted',
                'abp': result['abp'],
            })
        processed_windows.append(data)

In [ ]:
abp = [x['abp'] for x in processed_windows if x['status'] == 'predicted']
n = len(abp)

fig = make_subplots(rows=n, cols=1)
for i, sig in enumerate(abp):
    fig.add_scatter(y=sig, row=i+1, col=1)

fig

In [ ]:
_ = [print(x['snr']) for x in processed_windows]

In [ ]:
_ = [print(x['f0']) for x in processed_windows]

In [ ]:
_ = [print(x['beat_sim']) for x in processed_windows]

In [ ]:
_ = [print(x['notches']) for x in processed_windows]

In [ ]:
_ = [print(x['flat_lines']) for x in processed_windows]

In [ ]:
from database_tools.processing.detect import detect_flat_lines

x = processed_windows[1]['ppg']

pd.Series(x).plot()

In [ ]:
not detect_flat_lines(x, 10)